In [4]:
!pip install -q openai

In [5]:
!pip install tenacity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "./drive/MyDrive/SemEval/"

In [ ]:
!ls

In [1]:
import os
import re
import json
import time
import random
import config
import pandas as pd
import numpy as np
# from openai import OpenAI
# from tenacity import retry, stop_after_attempt, wait_fixed
# %run "Cleaning.ipynb"

In [62]:
# Define the function to fill the blank in the headline
@retry(stop=stop_after_attempt(3), wait=wait_fixed(20))
def fill_blank(news, masked_headline, prompt, client):
    # Construct the prompt by combining the news text and masked headline
    prompt = prompt.replace("<NEWS>", news).replace("<MASKED HEADLINE", masked_headline)
    messages = []
    messages.append(
        {"role": "system", "content": "You are a reasonable and accurate assistant"}
    )
    messages.append(
        {"role": "user", "content": prompt},
    )
    time.sleep(20)
    chat = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
    )
    reply = chat.choices[0].message.content
    # Extract the generated text from the response and return it
    return reply

In [63]:
class_to_id = {"copy":0, "trans":1, "paraphrase":2, "round":3, "subtract":4, "add":5, "span":6, "divide": 7, "multiply":8, "sround":9}

In [86]:
df = pd.read_csv('../Dev_Numerical_Reasoning_Cleaned.tsv', sep='\t')

In [87]:
def my_function(row):
  cal = row["calculation"]
  cal_extract = cal[:cal.index('(')].lower().strip()
  cal_id = class_to_id[cal_extract]
  return cal_id

In [88]:
df['calculation_id'] = df.apply(my_function, axis=1)

In [101]:
weights = [0.66, 0.17, 0.07, 0.03, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01]

In [104]:
np.array(weights).sum()

1.0

In [106]:
sample_df = pd.DataFrame()
for key, value in class_to_id.items():
  op_df = df[df['calculation_id'] == value].sample(n=int(weights[value] * 200), random_state=42)
  sample_df = pd.concat([sample_df, op_df])

In [107]:
# Calculate the weights for each value in the 'calculation_id' column
weights_ = sample_df['calculation_id'].value_counts(normalize=True)

In [108]:
weights_

0    0.66
1    0.17
2    0.07
3    0.03
4    0.02
5    0.01
6    0.01
7    0.01
8    0.01
9    0.01
Name: calculation_id, dtype: float64

In [ ]:
# Set up OpenAI API credentials
client = OpenAI(
    # This is the default and can be omitted
    api_key = config.API_KEY
)
# The prompt to feed into chatgpt
prompt = "Act as a news expert. I have a text of a news and its masked headline with mask token specified as [MASK]. The mask should be filled with a numerical value. you should just give me the numerical value to put instead of the [MASK].\nthe news is:\n<NEWS>\nthe masked headline is:\n<MASKED HEADLINE>.\nyour response should be in format of json with key of ans and value of the numerical answer."

In [ ]:
def find_answer(text: str) -> str:
  json.loads(text)['ans']
  text = text.replace(" ","")
  match = re.search(r"<MASK>=([0-9]*[/|\-|\\.]*[0-9]*)", text)
  if match:
      number = match.group(1)
      return number
  match = re.search(r"([0-9]+)", text)
  if match:
    number = match.group(1)
    return number
  return -1

In [ ]:
def test_on_chatgpt(sample, prompt, client, fill_blank, find_answer):
  # Fill the blank in the headline for each item in the sample
  sample["response"] = sample.apply(
      lambda row: fill_blank(row["news"], row["masked headline"], prompt, client), axis=1
  )
  sample["predicted_ans"] = sample["response"].apply(find_answer)
  # Calculate the accuracy of ChatGPT's predictions
  accuracy = (sample["ans"] == sample["predicted_ans"]).mean()
  return accuracy

In [ ]:
accuracy = test_on_chatgpt(sample_df, prompt, client, fill_blank, find_answer)

In [ ]:
sample_df[['news', 'masked headline', 'ans', 'predicted_ans', 'calculation', 'response']].to_csv('chatgpt_output_2.csv', index=False)

In [2]:
df = pd.read_csv("chatgpt-v2_output.csv")  

In [3]:
accuracy = (df["ans"] == df["predicted_ans"]).mean()

In [4]:
accuracy

0.775